<a href="https://colab.research.google.com/github/gray-james/post_generator/blob/main/instagram_generator_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instagram Post Generator - Complete Google Colab Notebook

This notebook contains all functionality from the Instagram Post Generator project.

## Features:
- Create single posts with text, author attribution, and illustrations
- Batch process multiple posts from JSON
- Create animated posts (fade-in and typewriter effects)
- Customizable styles and templates
- Various illustration types (person, brain, tree, heart, star, team, idea, growth)

## 1. Install Required Dependencies

In [1]:
# Install required packages
!pip install svgwrite pillow cairosvg imageio numpy

# For Google Colab, install system dependencies for cairosvg
!apt-get update
!apt-get install -y libcairo2-dev pkg-config python3-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Pa

## 2. Import Required Libraries

In [2]:
import svgwrite
from PIL import Image
import cairosvg
import json
from pathlib import Path
import textwrap
import math
import imageio
import numpy as np
import os
import argparse
import sys
from typing import Optional, Dict, Any, List
from google.colab import userdata


## 3. Style Configuration Setup

In [3]:
# Default style configuration
DEFAULT_STYLE = {
  "name": "default",
  "canvas": {
    "width": 1080,
    "height": 1080,
    "dpi": 300
  },
  "colors": {
    "background": "#FFFFFF",
    "primary_text": "#000000",
    "secondary_text": "#282626",
    "accent": "#FFD500",
    "illustration_primary": "#00965C",
    "illustration_secondary": "#ACE5CF",
    "pattern": "#F3F3F3"
  },
  "typography": {
    "heading": {
      "family": "Montserrat",
      "fallback": "sans-serif",
      "size": 72,
      "weight": "bold"
    },
    "body": {
      "family": "Noto Sans",
      "fallback": "Arial, sans-serif",
      "size": 48,
      "weight": "normal"
    },
    "accent": {
      "family": "Noto Sans",
      "fallback": "Arial, sans-serif",
      "size": 36,
      "weight": "bold"
    }
  },
  "patterns": {
    "enabled": True,
    "type": "dots",
    "opacity": 0.1,
    "spacing": 30
  },
  "layout": {
    "padding": 100,
    "line_spacing": 1.5,
    "max_width": 880
  }
}

# Custom dark style configuration
CUSTOM_STYLE = {
  "name": "custom",
  "canvas": {
    "width": 1080,
    "height": 1080,
    "dpi": 300
  },
  "colors": {
    "background": "#1a1a2e",
    "primary_text": "#eee",
    "secondary_text": "#aaa",
    "accent": "#f39c12",
    "illustration_primary": "#3498db",
    "illustration_secondary": "#2ecc71",
    "pattern": "#16213e"
  },
  "typography": {
    "heading": {
      "family": "Arial",
      "fallback": "sans-serif",
      "size": 84,
      "weight": "bold"
    },
    "body": {
      "family": "Helvetica",
      "fallback": "sans-serif",
      "size": 56,
      "weight": "normal"
    },
    "accent": {
      "family": "Georgia",
      "fallback": "serif",
      "size": 42,
      "weight": "bold"
    }
  },
  "patterns": {
    "enabled": True,
    "type": "grid",
    "opacity": 0.05,
    "spacing": 40
  },
  "layout": {
    "padding": 120,
    "line_spacing": 1.6,
    "max_width": 840
  }
}

## 4. Shape Library - Basic Illustrations

In [4]:
class ShapeLibrary:
    @staticmethod
    def create_person(dwg, center_x, center_y, size=100, color="#000000"):
        """Create simplified person illustration"""
        g = dwg.g()

        # Head
        head_radius = size * 0.15
        g.add(dwg.circle(center=(center_x, center_y - size * 0.35),
                         r=head_radius, fill=color))

        # Body (rounded rectangle)
        body_height = size * 0.4
        body_width = size * 0.3
        body_y = center_y - size * 0.2
        g.add(dwg.rect(insert=(center_x - body_width/2, body_y),
                       size=(body_width, body_height),
                       rx=body_width * 0.1, fill=color))

        # Arms
        arm_length = size * 0.3
        arm_width = size * 0.08
        # Left arm
        g.add(dwg.rect(insert=(center_x - body_width/2 - arm_width, body_y + body_height * 0.1),
                       size=(arm_width, arm_length),
                       rx=arm_width * 0.5, fill=color))
        # Right arm
        g.add(dwg.rect(insert=(center_x + body_width/2, body_y + body_height * 0.1),
                       size=(arm_width, arm_length),
                       rx=arm_width * 0.5, fill=color))

        # Legs
        leg_width = size * 0.1
        leg_height = size * 0.35
        leg_spacing = size * 0.05
        # Left leg
        g.add(dwg.rect(insert=(center_x - leg_width - leg_spacing/2, body_y + body_height),
                       size=(leg_width, leg_height),
                       rx=leg_width * 0.5, fill=color))
        # Right leg
        g.add(dwg.rect(insert=(center_x + leg_spacing/2, body_y + body_height),
                       size=(leg_width, leg_height),
                       rx=leg_width * 0.5, fill=color))

        return g

    @staticmethod
    def create_brain(dwg, center_x, center_y, size=100, color="#000000"):
        """Create simplified brain illustration using path"""
        g = dwg.g()

        # Scale factor
        s = size / 100

        # Brain outline path (simplified)
        path_data = f"""
        M {center_x - 40*s} {center_y}
        C {center_x - 40*s} {center_y - 30*s}, {center_x - 20*s} {center_y - 45*s}, {center_x} {center_y - 45*s}
        C {center_x + 20*s} {center_y - 45*s}, {center_x + 40*s} {center_y - 30*s}, {center_x + 40*s} {center_y}
        C {center_x + 40*s} {center_y + 20*s}, {center_x + 25*s} {center_y + 35*s}, {center_x} {center_y + 35*s}
        C {center_x - 25*s} {center_y + 35*s}, {center_x - 40*s} {center_y + 20*s}, {center_x - 40*s} {center_y}
        """

        # Main brain shape
        brain = dwg.path(d=path_data.strip(), fill=color, stroke='none') # Use strip() to remove leading/trailing whitespace
        g.add(brain)

        # Add center line for brain halves
        center_line = dwg.path(
            d=f"M {center_x} {center_y - 40*s} Q {center_x} {center_y}, {center_x} {center_y + 30*s}",
            fill='none',
            stroke=color,  # Use primary illustration color
            stroke_width=3*s
        )
        g.add(center_line)

        # Add some brain wrinkles
        wrinkle1 = dwg.path(
            d=f"M {center_x - 20*s} {center_y - 20*s} Q {center_x - 10*s} {center_y - 15*s}, {center_x - 15*s} {center_y - 5*s}",
            fill='none',
            stroke=color, # Use primary illustration color
            stroke_width=2*s
        )
        wrinkle2 = dwg.path(
            d=f"M {center_x + 20*s} {center_y - 20*s} Q {center_x + 10*s} {center_y - 15*s}, {center_x + 15*s} {center_y - 5*s}",
            fill='none',
            stroke=color, # Use primary illustration color
            stroke_width=2*s
        )
        g.add(wrinkle1)
        g.add(wrinkle2)

        return g

    @staticmethod
    def create_tree(dwg, center_x, center_y, size=100, trunk_color="#8B4513", leaves_color="#228B22"):
        """Create simplified tree illustration"""
        g = dwg.g()

        # Trunk
        trunk_width = size * 0.2
        trunk_height = size * 0.4
        trunk_y = center_y
        g.add(dwg.rect(insert=(center_x - trunk_width/2, trunk_y),
                       size=(trunk_width, trunk_height),
                       fill=trunk_color))

        # Leaves (circles for simple design)
        leaf_radius = size * 0.35
        # Bottom layer
        g.add(dwg.circle(center=(center_x, center_y - size * 0.1),
                        r=leaf_radius, fill=leaves_color))
        # Middle layer
        g.add(dwg.circle(center=(center_x - size * 0.15, center_y - size * 0.25),
                        r=leaf_radius * 0.8, fill=leaves_color))
        g.add(dwg.circle(center=(center_x + size * 0.15, center_y - size * 0.25),
                        r=leaf_radius * 0.8, fill=leaves_color))
        # Top layer
        g.add(dwg.circle(center=(center_x, center_y - size * 0.4),
                        r=leaf_radius * 0.7, fill=leaves_color))

        return g

    @staticmethod
    def create_heart(dwg, center_x, center_y, size=100, color="#FF0000"):
        """Create heart shape"""
        g = dwg.g()

        # Scale factor
        s = size / 100

        # Heart path - ensure it's a single line string
        path_data = (
            f"M {center_x} {center_y + 25*s} "
            f"C {center_x - 25*s} {center_y + 10*s}, {center_x - 40*s} {center_y - 10*s}, {center_x - 40*s} {center_y - 20*s} "
            f"C {center_x - 40*s} {center_y - 35*s}, {center_x - 20*s} {center_y - 45*s}, {center_x} {center_y - 30*s} "
            f"C {center_x + 20*s} {center_y - 45*s}, {center_x + 40*s} {center_y - 35*s}, {center_x + 40*s} {center_y - 20*s} "
            f"C {center_x + 40*s} {center_y - 10*s}, {center_x + 25*s} {center_y + 10*s}, {center_x} {center_y + 25*s}"
        )


        heart = dwg.path(d=path_data, fill=color, stroke='none')
        g.add(heart)

        return g

    @staticmethod
    def create_star(dwg, center_x, center_y, size=100, color="#FFD700", points=5):
        """Create star shape"""
        g = dwg.g()

        outer_radius = size * 0.5
        inner_radius = outer_radius * 0.4

        angle = 2 * math.pi / points

        points_list = []
        for i in range(points * 2):
            if i % 2 == 0:
                r = outer_radius
            else:
                r = inner_radius

            x = center_x + r * math.cos(i * angle / 2 - math.pi / 2)
            y = center_y + r * math.sin(i * angle / 2 - math.pi / 2)
            points_list.append((x, y))

        star = dwg.polygon(points=points_list, fill=color)
        g.add(star)

        return g

## 4a. Freepik API Call


### Load API Key

In [5]:
FREEPIK_API_KEY = userdata.get('FREEPIK_API_KEY')

### Search Term List

In [6]:
SEARCH_TERM_LIST = [
  "brain icon",
  "mood swing icon",
  "balance scale icon",
  "wave pattern icon",
  "calendar icon",
  "clock icon",
  "sleep icon",
  "sun and moon icon",
  "medication pill icon",
  "therapy session icon",
  "support group icon",
  "family icon",
  "friends circle icon",
  "doctor icon",
  "psychiatrist icon",
  "notebook icon",
  "mood chart icon",
  "graph line icon",
  "up and down arrow icon",
  "energy level icon",
  "lightbulb icon",
  "thought bubble icon",
  "emotion faces icon",
  "happy face icon",
  "sad face icon",
  "neutral face icon",
  "routine checklist icon",
  "daily planner icon",
  "exercise icon",
  "meditation icon",
  "breathing exercise icon",
  "healthy food icon",
  "water bottle icon",
  "bed icon",
  "alarm clock icon",
  "sunrise icon",
  "sunset icon",
  "moon phases icon",
  "weather icons set",
  "thermometer icon",
  "battery level icon",
  "shield icon",
  "heart icon",
  "brain waves icon",
  "neuron icon",
  "synapse icon",
  "dna helix icon",
  "genetics icon",
  "family tree icon",
  "stress icon",
  "warning sign icon",
  "alert icon",
  "information icon",
  "question mark icon",
  "exclamation mark icon",
  "checkmark icon",
  "x mark icon",
  "plus minus icon",
  "yin yang icon",
  "puzzle piece icon",
  "connection icon",
  "link chain icon",
  "broken chain icon",
  "bridge icon",
  "mountain peak icon",
  "valley icon",
  "roller coaster icon",
  "compass icon",
  "map icon",
  "journey path icon",
  "footsteps icon",
  "hand holding heart icon",
  "helping hands icon",
  "handshake icon",
  "phone call icon",
  "video call icon",
  "message bubble icon",
  "email icon",
  "emergency icon",
  "hospital icon",
  "ambulance icon",
  "first aid icon",
  "stethoscope icon",
  "clipboard icon",
  "prescription icon",
  "pharmacy icon",
  "timer icon",
  "stopwatch icon",
  "hourglass icon",
  "cycle arrows icon",
  "refresh icon",
  "repeat icon",
  "infinity icon",
  "target icon",
  "goal flag icon",
  "achievement medal icon",
  "star icon",
  "lightning bolt icon",
  "cloud icon",
  "rain cloud icon",
  "storm cloud icon",
  "rainbow icon",
  "umbrella icon",
  "anchor icon",
  "lifebuoy icon",
  "lighthouse icon",
  "compass rose icon",
  "steering wheel icon",
  "key icon",
  "lock unlock icon",
  "open door icon",
  "window icon",
  "light switch icon",
  "lamp icon",
  "candle icon",
  "fire icon",
  "water drop icon",
  "plant growing icon",
  "tree icon",
  "leaf icon",
  "flower icon",
  "butterfly icon",
  "bird flying icon",
  "feather icon",
  "bubbles icon",
  "balloon icon",
  "kite icon",
  "paper plane icon",
  "rocket icon",
  "stairs icon",
  "ladder icon",
  "elevator icon",
  "escalator icon",
  "bridge crossing icon",
  "road path icon",
  "crossroads icon",
  "traffic light icon",
  "stop sign icon",
  "direction arrow icon",
  "navigation icon",
  "milestone icon",
  "flag marker icon",
  "pin location icon",
  "home icon",
  "safe space icon",
  "comfort zone icon",
  "boundary fence icon",
  "filter icon",
  "magnifying glass icon",
  "microscope icon",
  "telescope icon",
  "binoculars icon",
  "eye icon",
  "ear listening icon",
  "speaking mouth icon",
  "raised hand icon",
  "open palm icon",
  "fist bump icon",
  "thumbs up icon",
  "peace sign icon"
]

### Main Code

In [7]:
# import requests
# import json

# url = "https://api.freepik.com/v1/resources"

# headers = {"x-freepik-api-key": FREEPIK_API_KEY} # Use the key loaded from userdata

# for term in SEARCH_TERM_LIST:
#   querystring = {
#   "order": "relevance",
#   "filters[ai-generated]": "false", # Assuming you don't want AI generated images, or set to "true" if you do
#   "term": SEARCH_TERM_LIST,
#   "limit": "1" # Increased limit to potentially get more results with SVGs
#   }


#   response = requests.get(url, headers=headers, params=querystring)
#   print(f"Status Code: {response.status_code}")

#   try:
#       response_data = response.json()
#       print(json.dumps(response_data, indent=4)) # Prettify and print the JSON response

#       svg_item_ids = []

#       # Navigate to data -> meta -> available_formats -> svg -> items -> id
#       for resource in response_data.get('data', []):
#           meta = resource.get('meta', {})
#           available_formats = meta.get('available_formats', {})
#           svg_format = available_formats.get('svg')

#           if svg_format and svg_format.get('items'):
#               for item in svg_format.get('items', []):
#                   if item.get('id') is not None:
#                       svg_item_ids.append(item['id'])

#       print(f"Found {len(svg_item_ids)} SVG item IDs:")
#       print(svg_item_ids)


#   except json.JSONDecodeError:
#       print("Could not decode JSON from response.")
#   except NameError:
#       print("Please ensure the FREEPIK_API_KEY and SEARCH_TERM_LIST variables are defined.")

In [8]:
svg_resource_ids = []

# Iterate through the data in the response
for resource in response_data.get('data', []):
    # Check if 'available_formats' and 'svg' exist and are not None
    if resource.get('meta', {}).get('available_formats', {}).get('svg') is not None:
        # Iterate through the SVG items
        for svg_item in resource['meta']['available_formats']['svg'].get('items', []):
            # Append the resource ID to the list
            svg_resource_ids.append(resource['id'])

print(f"Found {len(svg_resource_ids)} SVG resource IDs:")
print(svg_resource_ids)



NameError: name 'response_data' is not defined

In [ ]:
import json

# Assuming 'response' is the variable holding the response object from the previous cell
try:
    response_data = response.json()
    print(json.dumps(response_data, indent=4))
except json.JSONDecodeError:
    print("Could not decode JSON from response.")
except NameError:
    print("Please run the previous cell to get the response object.")

## 5. Composite Illustrations

In [9]:
class CompositeIllustrations:
    @staticmethod
    def create_team(dwg, center_x, center_y, size=150, color="#000000"):
        """Create team of people"""
        g = dwg.g()

        # Three people in a row
        positions = [(-size*0.35, 0), (0, -size*0.05), (size*0.35, 0)]
        sizes = [size*0.7, size*0.8, size*0.7]

        for (x_offset, y_offset), person_size in zip(positions, sizes):
            person = ShapeLibrary.create_person(
                dwg,
                center_x + x_offset,
                center_y + y_offset,
                size=person_size,
                color=color
            )
            g.add(person)

        return g

    @staticmethod
    def create_idea(dwg, center_x, center_y, size=150, brain_color="#FF6B6B", bulb_color="#FFD700"):
        """Create brain with lightbulb (idea concept)"""
        g = dwg.g()

        # Brain
        brain = ShapeLibrary.create_brain(dwg, center_x, center_y + size*0.1,
                                        size=size*0.8, color=brain_color)
        g.add(brain)

        # Lightbulb above brain (simplified)
        bulb_size = size * 0.3
        bulb_y = center_y - size * 0.4

        # Bulb
        bulb = dwg.circle(center=(center_x, bulb_y), r=bulb_size*0.5, fill=bulb_color)
        g.add(bulb)

        # Base
        base_width = bulb_size * 0.4
        base_height = bulb_size * 0.2
        base = dwg.rect(insert=(center_x - base_width/2, bulb_y + bulb_size*0.4),
                       size=(base_width, base_height), fill=brain_color)
        g.add(base)

        # Light rays
        for angle in [-30, -15, 0, 15, 30]:
            rad = math.radians(angle)
            x1 = center_x + bulb_size * 0.6 * math.sin(rad)
            y1 = bulb_y - bulb_size * 0.6 * math.cos(rad)
            x2 = center_x + bulb_size * 0.9 * math.sin(rad)
            y2 = bulb_y - bulb_size * 0.9 * math.cos(rad)

            ray = dwg.line(start=(x1, y1), end=(x2, y2),
                          stroke=bulb_color, stroke_width=3)
            g.add(ray)

        return g

    @staticmethod
    def create_growth(dwg, center_x, center_y, size=150):
        """Create growing plant/tree sequence"""
        g = dwg.g()

        # Three stages of growth
        positions = [(-size*0.35, 0), (0, 0), (size*0.35, 0)]
        tree_sizes = [size*0.4, size*0.6, size*0.8]

        for (x_offset, _), tree_size in zip(positions, tree_sizes):
            tree = ShapeLibrary.create_tree(
                dwg,
                center_x + x_offset,
                center_y,
                size=tree_size
            )
            g.add(tree)

        # Arrow between trees
        arrow_y = center_y + size * 0.2
        for i in range(2):
            start_x = center_x + positions[i][0] + size*0.1
            end_x = center_x + positions[i+1][0] - size*0.1

            arrow = dwg.path(
                d=f"M {start_x} {arrow_y} L {end_x} {arrow_y} L {end_x-10} {arrow_y-5} M {end_x} {arrow_y} L {end_x-10} {arrow_y+5}",
                stroke="#666666",
                stroke_width=2,
                fill="none"
            )
            g.add(arrow)

        return g

## 6. Post Generator Base Class

In [10]:
class PostGenerator:
    def __init__(self, style=None):
        if style is None:
            self.style = DEFAULT_STYLE
        elif isinstance(style, dict):
            self.style = style
        elif isinstance(style, str):
            # If it's a string, try to parse as JSON or use predefined styles
            if style == 'custom':
                self.style = CUSTOM_STYLE
            else:
                self.style = DEFAULT_STYLE
        else:
            self.style = DEFAULT_STYLE

        self.width = self.style['canvas']['width']
        self.height = self.style['canvas']['height']
        self.dpi = self.style['canvas']['dpi']

    def create_base_svg(self):
        """Create base SVG with background and optional patterns"""
        dwg = svgwrite.Drawing(size=(self.width, self.height))

        # Add background
        dwg.add(dwg.rect(insert=(0, 0), size=(self.width, self.height),
                         fill=self.style['colors']['background']))

        # Add pattern if enabled
        if self.style['patterns']['enabled']:
            self._add_pattern(dwg)

        return dwg

    def _add_pattern(self, dwg):
        """Add background pattern based on style settings"""
        pattern_type = self.style['patterns']['type']
        if pattern_type == 'dots':
            self._add_dot_pattern(dwg)
        elif pattern_type == 'lines':
            self._add_line_pattern(dwg)
        elif pattern_type == 'grid':
            self._add_grid_pattern(dwg)

    def _add_dot_pattern(self, dwg):
        """Create dot pattern background"""
        pattern_id = 'dotPattern'
        spacing = self.style['patterns']['spacing']

        pattern = dwg.defs.add(dwg.pattern(id=pattern_id, size=(spacing, spacing),
                                          patternUnits="userSpaceOnUse"))
        pattern.add(dwg.circle(center=(spacing/2, spacing/2), r=2,
                              fill=self.style['colors']['pattern'],
                              opacity=self.style['patterns']['opacity']))

        dwg.add(dwg.rect(insert=(0, 0), size=(self.width, self.height),
                        fill=f"url(#{pattern_id})"))

    def _add_line_pattern(self, dwg):
        """Create diagonal line pattern"""
        pattern_id = 'linePattern'
        spacing = self.style['patterns']['spacing']

        pattern = dwg.defs.add(dwg.pattern(id=pattern_id, size=(spacing, spacing),
                                          patternUnits="userSpaceOnUse"))
        pattern.add(dwg.line(start=(0, spacing), end=(spacing, 0),
                            stroke=self.style['colors']['pattern'],
                            stroke_width=1,
                            opacity=self.style['patterns']['opacity']))

        dwg.add(dwg.rect(insert=(0, 0), size=(self.width, self.height),
                        fill=f"url(#{pattern_id})"))

    def _add_grid_pattern(self, dwg):
        """Create grid pattern"""
        pattern_id = 'gridPattern'
        spacing = self.style['patterns']['spacing']

        pattern = dwg.defs.add(dwg.pattern(id=pattern_id, size=(spacing, spacing),
                                          patternUnits="userSpaceOnUse"))
        pattern.add(dwg.rect(insert=(0, 0), size=(spacing, spacing),
                            fill="none",
                            stroke=self.style['colors']['pattern'],
                            stroke_width=1,
                            opacity=self.style['patterns']['opacity']))

        dwg.add(dwg.rect(insert=(0, 0), size=(self.width, self.height),
                        fill=f"url(#{pattern_id})"))

    def get_text_style(self, style_type='body'):
        """Get text styling from configuration"""
        typography = self.style['typography'][style_type]
        return {
            'font-family': f"{typography['family']}, {typography['fallback']}",
            'font-size': typography['size'],
            'font-weight': typography.get('weight', 'normal')
        }

    def add_centered_text(self, dwg, text, y_position=None, style_type='body', color=None):
        """Add centered text with automatic wrapping"""
        if y_position is None:
            y_position = self.height / 2

        if color is None:
            color = self.style['colors']['primary_text' if style_type != 'accent' else 'secondary_text']

        text_style = self.get_text_style(style_type)
        font_size = text_style['font-size']
        max_width = self.style['layout']['max_width']

        # Calculate approximate character width (rough estimate)
        char_width = font_size * 0.6
        chars_per_line = int(max_width / char_width)

        # Wrap text
        lines = textwrap.wrap(text, width=chars_per_line)

        # Calculate total text height
        line_height = font_size * self.style['layout']['line_spacing']
        total_height = len(lines) * line_height

        # Start position for first line
        current_y = y_position - (total_height / 2) + font_size

        for line in lines:
            text_element = dwg.text(line,
                                  insert=(self.width / 2, current_y),
                                  text_anchor='middle',
                                  fill=color,
                                  **text_style)
            dwg.add(text_element)
            current_y += line_height

    def export_to_png(self, dwg, output_path):
        """Convert SVG to PNG"""
        svg_string = dwg.tostring()

        # Ensure output directory exists
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)

        cairosvg.svg2png(bytestring=svg_string.encode('utf-8'),
                         write_to=output_path,
                         output_width=self.width,
                         output_height=self.height,
                         dpi=self.dpi)
        print(f"Exported: {output_path}")
        return output_path

    def export_to_svg(self, dwg, output_path):
        """Export SVG to file"""
        # Ensure output directory exists
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)

        dwg.saveas(output_path)
        print(f"Exported: {output_path}")
        return output_path

## 7. Quote Template

In [11]:
class QuoteTemplate(PostGenerator):
    def __init__(self, style=None):
        super().__init__(style)
        self.template_name = "quote"

    def create_post(self, text, author=None, illustration=None, illustration_position='top', output_path="output/static/quote.png"):
        """Create a quote post with optional author and illustration"""
        dwg = self.create_base_svg()

        # Determine text position based on illustration
        if illustration and illustration_position == 'top':
            text_y = self.height * 0.6
            illustration_y = self.height * 0.25
        elif illustration and illustration_position == 'bottom':
            text_y = self.height * 0.4
            illustration_y = self.height * 0.75
        else:
            text_y = self.height * 0.5
            illustration_y = self.height * 0.3

        # Add illustration if specified
        if illustration:
            self._add_illustration(dwg, illustration, illustration_y)

        # Add main quote text
        self.add_centered_text(dwg, f'"{text}"', y_position=text_y, style_type='body')

        # Add author if provided
        if author:
            author_y = text_y + 100  # Adjust based on text height
            self.add_centered_text(dwg, f'— {author}',
                                 y_position=author_y,
                                 style_type='accent')

        # Export to PNG
        return self.export_to_svg(dwg, output_path)

    def _add_illustration(self, dwg, illustration_type, y_position):
        """Add illustration based on type"""
        x_center = self.width / 2
        illustration_size = 120

        # Get illustration color from style
        ill_color = self.style['colors'].get('illustration_primary', '#000000')

        if illustration_type == 'person':
            shape = ShapeLibrary.create_person(dwg, x_center, y_position,
                                             size=illustration_size, color=ill_color)
        elif illustration_type == 'brain':
            shape = ShapeLibrary.create_brain(dwg, x_center, y_position,
                                            size=illustration_size, color=ill_color)
        elif illustration_type == 'tree':
            shape = ShapeLibrary.create_tree(dwg, x_center, y_position,
                                           size=illustration_size)
        elif illustration_type == 'heart':
            shape = ShapeLibrary.create_heart(dwg, x_center, y_position,
                                            size=illustration_size, color=ill_color)
        elif illustration_type == 'star':
            shape = ShapeLibrary.create_star(dwg, x_center, y_position,
                                           size=illustration_size, color=ill_color)
        elif illustration_type == 'team':
            shape = CompositeIllustrations.create_team(dwg, x_center, y_position,
                                                      size=illustration_size*1.5, color=ill_color)
        elif illustration_type == 'idea':
            shape = CompositeIllustrations.create_idea(dwg, x_center, y_position,
                                                     size=illustration_size*1.5)
        elif illustration_type == 'growth':
            shape = CompositeIllustrations.create_growth(dwg, x_center, y_position,
                                                       size=illustration_size*1.5)
        else:
            return

        dwg.add(shape)

## 8. Batch Processor

In [12]:
class BatchProcessor:
    def __init__(self, template_class, style=None):
        self.template_class = template_class
        self.style = style

    def process_batch(self, posts_data, output_dir="output/static"):
        """Process multiple posts from list of dictionaries"""
        # Ensure output directory exists
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        results = []
        for i, post_data in enumerate(posts_data):
            # Generate filename
            base_name = post_data.get('filename', f'post_{i+1}')
            output_path = os.path.join(output_dir, f"{base_name}.svg")

            # Create template instance for each post
            template = self.template_class(self.style)

            # Remove filename from post_data if it exists
            post_data_copy = post_data.copy()
            post_data_copy.pop('filename', None)

            # Create post
            result_path = template.create_post(**post_data_copy, output_path=output_path)
            results.append(result_path)

        print(f"Batch processing complete. Generated {len(results)} posts.")
        return results

## 9. Animation Generator

In [13]:
class AnimationGenerator:
    def __init__(self, template_class, style=None):
        self.template_class = template_class
        self.style = style
        self.fps = 30
        self.duration = 3  # seconds

    def create_fade_in_animation(self, text, author=None, illustration=None, output_path="output/animated/fade.mp4"):
        """Create simple fade-in animation"""
        frames = []
        total_frames = int(self.fps * self.duration)

        # Ensure output directory exists
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)

        # Create full image
        template = self.template_class(self.style)
        temp_path = "temp_full.png"
        template.create_post(text=text, author=author, illustration=illustration, output_path=temp_path)

        # Load the full image
        full_image = Image.open(temp_path)
        full_array = np.array(full_image)

        # Create white background
        white_background = np.ones_like(full_array) * 255

        # Generate frames with increasing opacity
        for i in range(total_frames):
            alpha = i / total_frames
            frame = (1 - alpha) * white_background + alpha * full_array
            frames.append(frame.astype(np.uint8))

        # Save as video
        imageio.mimsave(output_path, frames, fps=self.fps)

        # Clean up temp file
        os.remove(temp_path)
        print(f"Animation saved to: {output_path}")

        return output_path

    def create_typewriter_animation(self, text, author=None, output_path="output/animated/typewriter.gif"):
        """Create typewriter text effect as GIF"""
        frames = []

        # Ensure output directory exists
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)

        # Calculate total characters
        total_chars = len(text)
        chars_per_frame = max(1, total_chars // 30)  # Aim for ~30 frames

        # Generate frames with progressive text
        current_text = ""
        for i in range(0, total_chars + 1, chars_per_frame):
            current_text = text[:i]

            # Create frame
            template = self.template_class(self.style)
            temp_path = f"temp_frame_{i}.png"
            template.create_post(text=current_text, author=author if i == total_chars else None,
                               output_path=temp_path)

            # Load and add frame
            frame = Image.open(temp_path)
            frames.append(frame)

            # Clean up temp file
            os.remove(temp_path)

        # Add final frame multiple times for pause
        if frames:
            for _ in range(10):
                frames.append(frames[-1])

        # Save as GIF
        if frames:
            frames[0].save(output_path, save_all=True, append_images=frames[1:],
                          duration=100, loop=0)
            print(f"Animation saved to: {output_path}")

        return output_path

## 10. Sample Batch Data

In [14]:
SAMPLE_BATCH_DATA = [
    {
        "text": "MYTH: Bipolar is just mood swings that everyone has",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_myth_1"
    },
    {
        "text": "FACT: Bipolar involves extreme episodes lasting weeks or months, with distinct periods of mania/hypomania and depression that significantly impair daily functioning",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_fact_1"
    },
    {
        "text": "MYTH: People with bipolar can't lead successful lives",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_myth_2"
    },
    {
        "text": "FACT: With proper treatment, many people with bipolar live fulfilling lives. The condition often comes with strengths like creativity, empathy, and resilience",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_fact_2"
    },
    {
        "text": "MYTH: Bipolar only affects your mood",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_myth_3"
    },
    {
        "text": "FACT: Bipolar affects energy, sleep, concentration, and judgment. During episodes, it can impact every aspect of a person's thinking and behavior",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_fact_3"
    },
    {
        "text": "MYTH: Mania is just feeling really happy and productive",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_myth_4"
    },
    {
        "text": "FACT: Mania can involve risky behavior, poor judgment, irritability, and even psychosis. It often leads to serious consequences and may require hospitalization",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_fact_4"
    },
    {
        "text": "MYTH: Medication is the only treatment needed",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_myth_5"
    },
    {
        "text": "FACT: Effective treatment combines medication, therapy, lifestyle changes, sleep regulation, and strong social support for best outcomes",
        "author": " ",
        "illustration": "star",
        "filename": "bipolar_fact_5"
    }
]

## 11. Main Functions - Single Post Creation

In [15]:
def create_single_post(text, author=None, illustration=None, style='default', output_path="single_post.png"):
    """Create a single Instagram post

    Args:
        text: The main text for the post
        author: Optional author attribution
        illustration: Optional illustration type ('person', 'brain', 'tree', 'heart', 'star', 'team', 'idea', 'growth')
        style: 'default' or 'custom' style
        output_path: Path to save the output image
    """
    style_dict = DEFAULT_STYLE if style == 'default' else CUSTOM_STYLE
    template = QuoteTemplate(style_dict)

    return template.create_post(
        text=text,
        author=author,
        illustration=illustration,
        output_path=output_path
    )

# Example usage
create_single_post(
    text="The only way to do great work is to love what you do",
    author="Steve Jobs",
    illustration="heart",
    output_path="example_post.png"
)

Exported: example_post.png


'example_post.png'

## 12. Batch Processing

In [16]:
def batch_process_posts(posts_data=None, style='default', output_dir="batch_output"):
    """Process multiple posts at once

    Args:
        posts_data: List of dictionaries with post information
        style: 'default' or 'custom' style
        output_dir: Directory to save output images
    """
    if posts_data is None:
        posts_data = SAMPLE_BATCH_DATA

    style_dict = DEFAULT_STYLE if style == 'default' else CUSTOM_STYLE
    processor = BatchProcessor(QuoteTemplate, style_dict)

    return processor.process_batch(posts_data, output_dir)


# Example usage
batch_process_posts(output_dir="batch_posts")

Exported: batch_posts/bipolar_myth_1.svg
Exported: batch_posts/bipolar_fact_1.svg
Exported: batch_posts/bipolar_myth_2.svg
Exported: batch_posts/bipolar_fact_2.svg
Exported: batch_posts/bipolar_myth_3.svg
Exported: batch_posts/bipolar_fact_3.svg
Exported: batch_posts/bipolar_myth_4.svg
Exported: batch_posts/bipolar_fact_4.svg
Exported: batch_posts/bipolar_myth_5.svg
Exported: batch_posts/bipolar_fact_5.svg
Batch processing complete. Generated 10 posts.


['batch_posts/bipolar_myth_1.svg',
 'batch_posts/bipolar_fact_1.svg',
 'batch_posts/bipolar_myth_2.svg',
 'batch_posts/bipolar_fact_2.svg',
 'batch_posts/bipolar_myth_3.svg',
 'batch_posts/bipolar_fact_3.svg',
 'batch_posts/bipolar_myth_4.svg',
 'batch_posts/bipolar_fact_4.svg',
 'batch_posts/bipolar_myth_5.svg',
 'batch_posts/bipolar_fact_5.svg']

## 13. Animation Creation

In [17]:
def create_animated_post(text, author=None, illustration=None, effect='fade', style='default', output_path=None):
    """Create an animated Instagram post

    Args:
        text: The main text for the post
        author: Optional author attribution
        illustration: Optional illustration type
        effect: 'fade' for fade-in effect or 'typewriter' for typewriter effect
        style: 'default' or 'custom' style
        output_path: Path to save the output animation
    """
    style_dict = DEFAULT_STYLE if style == 'default' else CUSTOM_STYLE
    animator = AnimationGenerator(QuoteTemplate, style_dict)

    if effect == 'fade':
        if output_path is None:
            output_path = "fade_animation.mp4"
        return animator.create_fade_in_animation(
            text=text,
            author=author,
            illustration=illustration,
            output_path=output_path
        )
    elif effect == 'typewriter':
        if output_path is None:
            output_path = "typewriter_animation.gif"
        return animator.create_typewriter_animation(
            text=text,
            author=author,
            output_path=output_path
        )

# Example usage - Fade animation
create_animated_post(
    text="Innovation distinguishes between a leader and a follower",
    author="Steve Jobs",
    illustration="idea",
    effect="fade",
    output_path="fade_example.mp4"
)

# Example usage - Typewriter animation
create_animated_post(
    text="Stay hungry, stay foolish",
    author="Steve Jobs",
    effect="typewriter",
    output_path="typewriter_example.gif"
)

Exported: temp_full.png


UnidentifiedImageError: cannot identify image file 'temp_full.png'

## 14. Interactive Post Creator

In [18]:
# Interactive widget for Google Colab
def interactive_post_creator():
    """Interactive interface for creating posts in Google Colab"""
    print("=" * 50)
    print("Instagram Post Generator - Interactive Mode")
    print("=" * 50)

    # Get user input
    text = input("Enter your post text: ")
    author = input("Enter author name (or press Enter to skip): ")

    print("\nAvailable illustrations:")
    illustrations = ['none', 'person', 'brain', 'tree', 'heart', 'star', 'team', 'idea', 'growth']
    for i, ill in enumerate(illustrations):
        print(f"{i}. {ill}")

    ill_choice = input("Select illustration number (default: 0): ")
    illustration = None if not ill_choice or ill_choice == '0' else illustrations[int(ill_choice)]

    print("\nAvailable styles:")
    print("1. Default (light)")
    print("2. Custom (dark)")
    style_choice = input("Select style (1 or 2, default: 1): ")
    style = 'custom' if style_choice == '2' else 'default'

    print("\nPost type:")
    print("1. Static image")
    print("2. Fade-in animation")
    print("3. Typewriter animation")
    type_choice = input("Select type (1, 2, or 3, default: 1): ")

    # Create the post
    if type_choice == '2':
        output = create_animated_post(
            text=text,
            author=author if author else None,
            illustration=illustration,
            effect='fade',
            style=style,
            output_path="interactive_fade.mp4"
        )
    elif type_choice == '3':
        output = create_animated_post(
            text=text,
            author=author if author else None,
            illustration=illustration,
            effect='typewriter',
            style=style,
            output_path="interactive_typewriter.gif"
        )
    else:
        output = create_single_post(
            text=text,
            author=author if author else None,
            illustration=illustration,
            style=style,
            output_path="interactive_post.png"
        )

    print(f"\n✅ Post created successfully: {output}")

    # Display the image in Colab
    from IPython.display import Image as IPImage, display
    if output.endswith('.png'):
        display(IPImage(output))

    return output

# Run the interactive creator
# interactive_post_creator()

## 15. Quick Start Examples

In [19]:
# Quick examples to get started

# 1. Simple text post
create_single_post(
    text="Hello World!",
    output_path="simple_post.png"
)

# 2. Quote with author and illustration
create_single_post(
    text="Be yourself; everyone else is already taken",
    author="Oscar Wilde",
    illustration="star",
    output_path="quote_post.png"
)

# 3. Dark theme post
create_single_post(
    text="Code is poetry",
    illustration="brain",
    style="custom",
    output_path="dark_theme_post.png"
)

print("✅ All example posts created!")

Exported: simple_post.png
Exported: quote_post.png
Exported: dark_theme_post.png
✅ All example posts created!


## 16. Utility Functions

In [20]:
def display_all_illustrations():
    """Display all available illustrations in a grid"""
    illustrations = ['person', 'brain', 'tree', 'heart', 'star', 'team', 'idea', 'growth']

    for ill in illustrations:
        create_single_post(
            text=f"Illustration: {ill.title()}",
            illustration=ill,
            output_path=f"illustration_{ill}.png"
        )

    print(f"Created {len(illustrations)} illustration examples")

def create_style_comparison():
    """Create the same post in both styles for comparison"""
    text = "Design is not just what it looks like. Design is how it works."
    author = "Steve Jobs"

    # Default style
    create_single_post(
        text=text,
        author=author,
        illustration="idea",
        style="default",
        output_path="style_default.png"
    )

    # Custom style
    create_single_post(
        text=text,
        author=author,
        illustration="idea",
        style="custom",
        output_path="style_custom.png"
    )

    print("Created style comparison posts")

# Run examples
# display_all_illustrations()
# create_style_comparison()

## Complete! 🎉

This notebook contains all the functionality from your Instagram Post Generator project. You can now:

1. **Create single posts** with custom text, authors, and illustrations
2. **Batch process** multiple posts from data
3. **Create animations** with fade-in or typewriter effects
4. **Use different styles** (light/dark themes)
5. **Choose from 8 illustration types**

To use in Google Colab:
1. Upload this notebook to Google Colab
2. Run the installation cell first
3. Then run all the class definition cells
4. Use the example functions or create your own posts!

All images will be saved to the Colab environment and can be downloaded.